In [1]:
import importlib
import ifimage_tools
import matplotlib.pyplot as plt
import importlib
import ifimage_tools
import matplotlib.pyplot as plt
importlib.reload(ifimage_tools)
from ifimage_tools import *
import pandas as pd
import os
from itertools import islice
import numpy as np
from stardist.matching import matching
import cv2
import pickle

In [ ]:
with open("dataset.pkl", "rb") as f:
    dataset = pickle.load(f)
print("Dataset loaded from dataset.pkl")
id= pd.read_csv("/Users/macbookair/PycharmProjects/ifimage/best_annotated_images/best_samples_report.csv")

In [ ]:
dataset.samples.items()

In [4]:
# for i in range(len(id)):
#     sample=dataset.samples[f"{id.iloc[i,1]}"]
#     draw_overlap(sample.cellbodies_mask,sample.cyto_positive_masks["cellpose2chan"],
#                  title=f"ID:{id.iloc[i,1]} {id.iloc[i,0].upper()} mAP={id.iloc[i,2]:.3f}",
#                  display_legend=True)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
from stardist.matching import matching

info = pd.read_csv("/Users/macbookair/PycharmProjects/ifimage/best_annotated_images/best_samples_report.csv")

fig, axs = plt.subplots(len(info), 4, figsize=(16, 4 * len(info)))

for i in range(len(info)):
    sample_id = info.iloc[i,1]
    sample = dataset.samples[str(sample_id)]
    celltype = sample.celltype
    
    cell_body_ann = sample.cellbodies_mask
    cell_body_pred = sample.cyto_positive_masks["cellpose2chan"]
    nuclei_ann = sample.dapi_multi_mask
    nuclei_pred = sample.masks["cyto3"]
    
    # Calculate IoU using stardist.matching
    iou_body = calculate_overall_iou(cell_body_ann, cell_body_pred)
    iou_nuclei = calculate_overall_iou(nuclei_ann, nuclei_pred)

    # Display images with celltype and sample_id labels
    # Show marker image in green channel
    axs[i, 0].imshow(sample.marker, cmap='Greens')
    axs[i, 0].axis('off')
    axs[i, 0].text(5, 20, f"{celltype} {sample_id}", color='white', fontsize=10, backgroundcolor='black')
    
    axs[i, 1].imshow(cell_body_pred, cmap="plasma")
    axs[i, 1].axis('off')
    axs[i, 1].text(5, 20, f"IoU: {iou_body:.2f}", color='white', fontsize=10, backgroundcolor='black')
    
    # Show dapi image in blue channel
    axs[i, 2].imshow(sample.dapi, cmap='Blues')
    axs[i, 2].axis('off')
    
    axs[i, 3].imshow(nuclei_pred, cmap='plasma')
    axs[i, 3].axis('off')
    axs[i, 3].text(5, 20, f"IoU: {iou_nuclei:.2f}", color='white', fontsize=10, backgroundcolor='black')

col_titles = ['Cell Body: Annotation', 'Prediction', 'Nuclei: Annotation', 'Prediction']
for ax, col in zip(axs[0], col_titles):
    ax.set_title(col, fontsize=14)

for ax, row in zip(axs[:, 0], info.iloc[:,0]):
    ax.set_ylabel(row.capitalize(), fontsize=14)

plt.tight_layout()
plt.savefig("summary_panel.pdf")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from stardist.matching import matching

info = pd.read_csv("/Users/macbookair/PycharmProjects/ifimage/best_annotated_images/best_samples_report.csv")

# Create figure with number of rows matching comparing_ids length
comparing_ids=["15972","1120","12779","8406jan22","4201"]
fig, axs = plt.subplots(len(comparing_ids), 4, figsize=(16, 4 * len(comparing_ids)))

# Use enumerate to get integer indices for axs access
for idx, sample_id in enumerate(comparing_ids):
    sample = dataset.samples[sample_id]
    celltype = sample.celltype
    
    cell_body_ann = sample.cellbodies_multimask
    cell_body_pred = sample.cyto_positive_masks["cellpose2chan"]
    nuclei_ann = sample.dapi_multi_mask
    nuclei_pred = sample.masks["cyto3"]
    
    # Calculate IoU using stardist.matching
    iou_body = calculate_overall_iou(cell_body_ann, cell_body_pred)
    iou_nuclei = calculate_overall_iou(nuclei_ann, nuclei_pred)

    # Display images with celltype and sample_id labels
    # Show marker image in green channel
    axs[idx, 0].imshow(sample.cellbodies_multimask, 
                       #cmap='Greens'
                       )
    axs[idx, 0].axis('off')
    axs[idx, 0].text(5, 20, f"{celltype} {sample_id}", color='white', fontsize=10, backgroundcolor='black')
    
    axs[idx, 1].imshow(cell_body_pred, cmap="plasma")
    axs[idx, 1].axis('off')
    axs[idx, 1].text(5, 20, f"IoU: {iou_body:.2f}", color='white', fontsize=10, backgroundcolor='black')
    
    # Show dapi image in blue channel
    axs[idx, 2].imshow(sample.dapi, cmap='Blues')
    axs[idx, 2].axis('off')
    
    # Convert nuclei_pred to float if it's object dtype
    if nuclei_pred.dtype == np.object_:
        nuclei_pred = nuclei_pred.astype(np.float32)
    axs[idx, 3].imshow(nuclei_pred, cmap='plasma')
    axs[idx, 3].axis('off')
    axs[idx, 3].text(5, 20, f"IoU: {iou_nuclei:.2f}", color='white', fontsize=10, backgroundcolor='black')

col_titles = ['Cell Body: Annotation', 'Prediction', 'Nuclei: Annotation', 'Prediction']
for ax, col in zip(axs[0], col_titles):
    ax.set_title(col, fontsize=14)

plt.tight_layout()
plt.savefig("summary_panel.pdf")
plt.show()

In [3]:
from preprocessing import *

In [7]:
# Convert ROI zip files to masks and save as numpy arrays
dapi_mask = rois_to_mask("/Users/macbookair/Downloads/ddd/GFAP_RoiSet_ALLDAPI_Final.zip", 1388, 1040)
np.save("gfap_4201_dapimultimask.npy", dapi_mask)

cellbodies_mask = rois_to_mask("/Users/macbookair/Downloads/ddd/GFAP_RoiSet_CellBodies_Final.zip", 1388, 1040) 
np.save("gfap_4201_cellbodiesmultimask.npy", cellbodies_mask)

ROI '0013-0263.roi' drawn with label 1.
ROI '0011-0070.roi' drawn with label 2.
ROI '0007-0250.roi' drawn with label 3.
ROI '0025-0212.roi' drawn with label 4.
ROI '0038-0207.roi' drawn with label 5.
ROI '0056-0096.roi' drawn with label 6.
ROI '0077-0112.roi' drawn with label 7.
ROI '0088-0108.roi' drawn with label 8.
ROI '0089-0130.roi' drawn with label 9.
ROI '0098-0156.roi' drawn with label 10.
ROI '0108-0127.roi' drawn with label 11.
ROI '0108-0069.roi' drawn with label 12.
ROI '0127-0081.roi' drawn with label 13.
ROI '0107-0047.roi' drawn with label 14.
ROI '0122-0052.roi' drawn with label 15.
ROI '0133-0069.roi' drawn with label 16.
ROI '0139-0077.roi' drawn with label 17.
ROI '0146-0109.roi' drawn with label 18.
ROI '0242-0018.roi' drawn with label 19.
ROI '0257-0046.roi' drawn with label 20.
ROI '0289-0047.roi' drawn with label 21.
ROI '0296-0029.roi' drawn with label 22.
ROI '0336-0019.roi' drawn with label 23.
ROI '0366-0031.roi' drawn with label 24.
ROI '0376-0011.roi' drawn